In [1]:
import logging

# Libraries

In [2]:
import pandas as pd
from collections import defaultdict
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

# Load Crentials

In [3]:
from dotenv import dotenv_values

config = dotenv_values(".env")
BEARER_TOKEN = config['BEARER_TOKEN_2']

In [4]:
logging.basicConfig(filename='App_LikedTweets_ServerStatus.log', encoding='utf-8', level=logging.ERROR, force=True)

# Liked Tweets

In [5]:
import requests
import os
import json

In [6]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def main(url, params):
    json_response = connect_to_endpoint(url, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

In [7]:
def expand_dict_object(object_dict: dict, column_prefix: str,key_names_list: list = []) -> dict:
    """Return key,value pairs from dict. Returns selected keys in key_names_list or all if key_names_list is []

    Args:
        object_dict (dict)
        key_names_list (list)
    Returns:
        dict
    """
    result_dict = defaultdict(list)
    for key_name, value_name in object_dict.items():
        column_name = f"{column_prefix}_{key_name}"
        if not key_names_list:
            result_dict[column_name] = value_name
        elif key_name in key_names_list:
            result_dict[column_name] = value_name
    return result_dict

In [8]:
def get_list_of_items(list_of_dict: list, key_name: str) -> list:
    if not list_of_dict: # If list_of_dict is None
        return []
    return [entity[key_name] for entity in list_of_dict]

In [9]:
def parse_entities(tweet_entities: dict, object_dict: dict, prefix: str = '') -> dict:
    entities_dict = defaultdict(list)

    # Retrieve Entities Objects
    for object_name, key_name in object_dict.items():
        column_name = f"{prefix}{object_name}_list"
        # print(object_name)
        entities_dict[column_name] = get_list_of_items(tweet_entities.get(object_name), key_name)
    return entities_dict
    # entities_dict['hashtags_list'] = get_list_of_items(tweet_entities['hashtags'], 'tag')
    # entities_dict['urls_list'] = get_list_of_items(tweet_entities['urls'], 'expanded_url')

In [10]:
usernames_sample  = pd.read_csv("data/unique_1500users.csv")

In [14]:
# Get users Ids

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {
}
# query_params = {}
search_url = "https://api.twitter.com/2/users/by/username/CarlsbadDodger"
# search_url="https://api.twitter.com/2/users/by/username/TeamYouTube"
# Snowstorm = 3001806479
# Team Youtube
output_tweets = main(search_url, query_params)


In [31]:
usernames_sample['screen_name'][11:]

11          davidgerard
12          hu_YUZHNOYE
13           tmbrown327
14      nekoyasumi_club
15           jinjinhnka
             ...       
1495        cloudboyebk
1496    KhogendraRupini
1497    LigiaMauraCosta
1498       EdgarAguirre
1499         Mark738eis
Name: screen_name, Length: 1489, dtype: object

In [65]:
# CarronMrs

for idx, name in enumerate(usernames_sample['screen_name']):
    if name == 'EdgarAguirre':
        break


In [66]:
idx

1498

In [68]:
BEARER_TOKEN = config['BEARER_TOKEN_7']

In [74]:
pd.DataFrame(set(list_of_ids), columns=['user_id']).to_csv('data/ids_1500.csv', index=False)

In [70]:
for user_name in usernames_sample['screen_name'][1498:]:
    print(user_name)
    # break
    search_url = f"https://api.twitter.com/2/users/by/username/{user_name}"
    # search_url="https://api.twitter.com/2/users/by/username/TeamYouTube"
    # Snowstorm = 3001806479
    # Team Youtube
    output_tweets = main(search_url, query_params)
    try:
        list_of_ids.append(output_tweets['data']['id'])
    except:
        pass

EdgarAguirre
Mark738eis


In [21]:
output_tweets

{'errors': [{'value': 'realkellerwade',
   'detail': 'Could not find user with username: [realkellerwade].',
   'title': 'Not Found Error',
   'resource_type': 'user',
   'parameter': 'username',
   'resource_id': 'realkellerwade',
   'type': 'https://api.twitter.com/2/problems/resource-not-found'}]}

In [10]:
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {
    'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
    'tweet.fields': 'created_at,public_metrics,entities,lang,possibly_sensitive,reply_settings,source,in_reply_to_user_id,geo,referenced_tweets',
    'expansions': 'author_id',
}
# query_params = {}
search_url = "https://api.twitter.com/2/users/3031071234/liked_tweets"
# search_url="https://api.twitter.com/2/users/by/username/TeamYouTube"
# Snowstorm = 3001806479
# Team Youtube
output_tweets = main(search_url, query_params)


# Retrieve Liked Tweets from users AVAX

In [23]:
path = './avax_data'
dir_list = os.listdir(path)

In [24]:
df = pd.read_csv(f'{path}/{dir_list[0]}')

/var/folders/sv/7czjs3z54yd54jz8j2ktmygc0000gn/T/ipykernel_21245/2770470333.py:1: DtypeWarning: Columns (6,7,8,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{path}/{dir_list[0]}')


In [28]:
users_id_list = set(df['userid'])

In [25]:
json_tweets_columns = [
        'id', 'author_id', 'possibly_sensitive', 'edit_history_tweet_ids', 'lang',
        'source', 'reply_settings', 'text', 'created_at'
    ]

tweet_columns = [
    'possibly_sensitive', 'text', 'source', 'id', 'created_at', 'lang', 'reply_settings', 'author_id', 'edit_history_tweet_ids', 
    'hashtags_list', 'urls_list', 'public_metrics_retweet_count', 'public_metrics_reply_count', 'public_metrics_like_count', 
    'public_metrics_quote_count', 'pagination_token', 'current_time'
]

In [26]:
sleep = 300 # in seconds
N = 5 # Number of retries. Maxium retry time = N*sleep (s)

In [27]:
current_time = datetime.now()

date_str = f'{current_time.year}-{current_time.month:02d}-{current_time.day:02d}'
time_str = f'{current_time.hour:02d}-{current_time.minute:02d}'

In [53]:
# for batch_of_users, batch_csv in enumerate(dir_list):
#     df = pd.read_csv(batch_csv)
#     logging.info(f"Read {batch_csv}")
batch_of_users = 0 # 0.csv
tweets_list_for_dataframe = []
users_list_for_dataframe = []

query_params = {
    'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
    'tweet.fields': 'created_at,public_metrics,entities,lang,possibly_sensitive,reply_settings,source,in_reply_to_user_id,geo,referenced_tweets',
    'expansions': 'author_id',
}

hast_next_token = True

for userid in list(users_id_list): # There are 200 users id
    print(userid)
    while hast_next_token:
        print('request')
        tweets_list_for_dataframe = []
        users_list_for_dataframe = []
        for attempt in range(N):
            print(attempt)
            try:
                # Query Data from API
                query_params['query'] = f'(from:{userid})'
                output_tweets = main(search_url, query_params)
                logging.info(f'Twitter URL: {search_url}')
                logging.info(f'Params: {query_params}')

                # If no data found:
                if not output_tweets.get('data'):
                    logging.warning(f'No data found for userId {userid}')
                    # continue
                    hast_next_token = False
                    break
                next_token = {}
                next_token['pagination_token'] = output_tweets['meta'].get('next_token')
                if next_token['pagination_token']:
                    query_params['pagination_token'] = next_token['pagination_token']
                    hast_next_token = True
                else:
                    hast_next_token = False

                for tweet in output_tweets['data']:
                    # Get new columns
                    if not tweet.get('entities'):
                        entities_dict = parse_entities({}, {'hashtags': 'tag', 'urls': 'expanded_url'})
                    else:
                        entities_dict = parse_entities(tweet['entities'], {'hashtags': 'tag', 'urls': 'expanded_url'})

                    if not tweet.get('public_metrics'):
                        public_metrics_dict = expand_dict_object({}, 'public_metrics')
                    else:
                        public_metrics_dict = expand_dict_object(tweet['public_metrics'], 'public_metrics')

                    # Filter out unwanted columns
                    tweet = {key: tweet[key] for key in tweet.keys() if key in json_tweets_columns}

                    # Combine dicts
                    tweet = {**tweet, **entities_dict, **public_metrics_dict, **next_token, 'current_time': datetime.now()}
                    # tweet = sorted(tweet.items(), key=lambda kv: kv[1])
                    tweets_list_for_dataframe.append(tweet)

# for userid in list(users_id_list): # There are 200 users id
#     while hast_next_token:
#         for attempt in range(N):
#             try:
#                 # Query Data from API
#                 search_url = f"https://api.twitter.com/2/users/{userid}/liked_tweets"
#                 output_tweets = main(search_url, query_params)
#                 logging.info(f'Twitter URL: {search_url}')
#                 logging.info(f'Params: {query_params}')

#                 # If no data found:
#                 if not output_tweets.get('data'):
#                     logging.warning(f'No data found for userId {userid}')
#                     continue

#                 next_token = {}
#                 next_token['pagination_token'] = output_tweets['meta'].get('next_token')
#                 if next_token['pagination_token']:
#                     query_params['pagination_token'] = next_token['pagination_token']
#                     hast_next_token = True
#                 else:
#                     hast_next_token = False

#                 for tweet in output_tweets['data']:
#                     # Get new columns
#                     if not tweet.get('entities'):
#                         entities_dict = parse_entities({}, {'hashtags': 'tag', 'urls': 'expanded_url'})
#                     else:
#                         entities_dict = parse_entities(tweet['entities'], {'hashtags': 'tag', 'urls': 'expanded_url'})
                    
#                     if not tweet.get('public_metrics'):
#                         public_metrics_dict = expand_dict_object({}, 'public_metrics')
#                     else:
#                         public_metrics_dict = expand_dict_object(tweet['public_metrics'], 'public_metrics')

#                     # Filter out unwanted columns
#                     tweet = {key: tweet[key] for key in tweet.keys() if key in json_tweets_columns}

#                     # Combine dicts
#                     tweet = {**tweet, **entities_dict, **public_metrics_dict, **next_token, 'current_time': datetime.now()}
#                     tweets_list_for_dataframe.append(tweet)

            except Exception as err:
                if err.args[0] == 429: # If ERROR = 429 (Too Many Requests, wait for retry) 
                    logging.info(f'Error 429. Sleep: {sleep}')
                    time.sleep(sleep)
                    # continue
                    raise
                else:
                    raise
            break

1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4539ky98xkteilwrcj3etufe2ckx'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4803qh0y60v44vlfxy97j0k2addk'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4803k2jgno90eq1scih76won6tdf'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4803dp9cefr7ofnms8qi5sbzzbjt'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4544d7d2hp0y1dfuqowfjccvyfjn'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4544b1fjuh77xwxj6gf0scrubo6t'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw45444nuoprplk3bfc8ijzy2noojd'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4543yakkgjgfb6bv57wksw39ewqd'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw4543tzbg8ekcqtnm9m4xxv30e69k'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw453p3b23ttu6s2h8rcz6hrwu9po4'}
1188283809826869248
{'pagination_token': '7140dibdnow9c7btw453oz2ifjcaznzl8dbrg83jhcuno'}


KeyboardInterrupt: 

In [2]:
from datetime import datetime

In [9]:
datetime.now().minute/2 < 20

False

In [5]:
1 if datetime.now().minute/2 < 20 else 0

1

In [32]:
import logging

import pandas as pd
from collections import defaultdict
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import os
import requests
import os
import json

# logging.basicConfig(filename='App_LikedTweets_ServerStatus.log', encoding='utf-8', level=logging.ERROR, force=True)

# Load Credentials
from dotenv import dotenv_values

config = dotenv_values(".env")
BEARER_TOKEN_2 = config['BEARER_TOKEN_2']
BEARER_TOKEN_3 = config['BEARER_TOKEN_3']

BEARER_TOKEN = BEARER_TOKEN_2 if datetime.now().minute/2 < 20 else BEARER_TOKEN_3

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def main(url, params):
    json_response = connect_to_endpoint(url, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

def get_list_of_items(list_of_dict: list, key_name: str) -> list:
    if not list_of_dict: # If list_of_dict is None
        return []
    return [entity[key_name] for entity in list_of_dict]

def parse_entities(tweet_entities: dict, object_dict: dict, prefix: str = '') -> dict:
    entities_dict = defaultdict(list)

    # Retrieve Entities Objects
    for object_name, key_name in object_dict.items():
        column_name = f"{prefix}{object_name}_list"
        # print(object_name)
        entities_dict[column_name] = get_list_of_items(tweet_entities.get(object_name), key_name)
    return entities_dict
    # entities_dict['hashtags_list'] = get_list_of_items(tweet_entities['hashtags'], 'tag')
    # entities_dict['urls_list'] = get_list_of_items(tweet_entities['urls'], 'expanded_url')

def expand_dict_object(object_dict: dict, column_prefix: str,key_names_list: list = []) -> dict:
    """Return key,value pairs from dict. Returns selected keys in key_names_list or all if key_names_list is []

    Args:
        object_dict (dict)
        key_names_list (list)
    Returns:
        dict
    """
    result_dict = defaultdict(list)
    for key_name, value_name in object_dict.items():
        column_name = f"{column_prefix}_{key_name}"
        if not key_names_list:
            result_dict[column_name] = value_name
        elif key_name in key_names_list:
            result_dict[column_name] = value_name
    return result_dict

if __name__ == '__main__':
    json_tweets_columns = [
        'id', 'author_id', 'possibly_sensitive', 'edit_history_tweet_ids', 'lang',
        'source', 'reply_settings', 'text', 'created_at'
    ]

    json_users_columns = [
        'id', 'name', 'username', 'location', 'url', 'created_at', 'username',
        'profile_image_url', 'profile_image_url', 'verified', 'description',
        'protected'
    ]
    user_columns = [
        'profile_image_url', 'username', 'protected', 'name', 'id', 'description', 
        'created_at', 'verified', 'location', 'url_urls_list', 'description_hashtags_list', 'description_urls_list', 'description_mentions_list',
        'description_cashtags_list', 'public_metrics_followers_count', 'public_metrics_following_count', 'public_metrics_tweet_count', 
        'public_metrics_listed_count', 'current_time'
    ]

    tweet_columns = [
        'possibly_sensitive', 'text', 'source', 'id', 'created_at', 'lang', 'reply_settings', 'author_id', 'edit_history_tweet_ids', 
        'hashtags_list', 'urls_list', 'public_metrics_retweet_count', 'public_metrics_reply_count', 'public_metrics_like_count', 
        'public_metrics_quote_count', 'pagination_token', 'current_time'
    ]
    # Compute Start and End Time with 6 month interval

    current_time = datetime.now()

    date_str = f'{current_time.year}-{current_time.month:02d}-{current_time.day:02d}'
    time_str = f'{current_time.hour:02d}-{current_time.minute:02d}'

    sleep = 300 # in seconds
    N = 5 # Number of retries. Maxium retry time = N*sleep (s)

    # path = '/project/ll_774_951/SMHabits/avax_project_data/random_user_tweets'
    path = 'avax_data'
    dir_list = os.listdir(path)

    for batch_of_users, batch_csv in enumerate(dir_list):
        df = pd.read_csv(f"{path}/{batch_csv}")
        logging.info(f"Read {batch_csv}")

        # Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
        # expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
        query_params = {
            'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
            'tweet.fields': 'created_at,public_metrics,entities,lang,possibly_sensitive,reply_settings,source,in_reply_to_user_id,geo,referenced_tweets',
            'expansions': 'author_id',
        }

        hast_next_token = True
        users_id_list = set(df['userid'])
        # Create CSV file
        if not os.path.exists(f"data/liked-tweets-{batch_of_users}.csv"):
            with open(f"data/liked-tweets-{batch_of_users}.csv", "a+", encoding='utf-8') as f:
                f.write(", ".join(tweet_columns) + "\n")
        for userid in list(users_id_list): # There are 200 users id
            print(userid)
            hast_next_token = True
            while hast_next_token:
                print(hast_next_token)
                tweets_list_for_dataframe = []
                users_list_for_dataframe = []
                for attempt in range(N):
                    print(attempt)
                    try:
                        # Query Data from API
                        search_url = f"https://api.twitter.com/2/users/{userid}/liked_tweets"

                        # If user data has already been collected, skip
                        try:
                            save_csv = True
                            tweets_history = set(pd.read_csv('data/liked-tweets-0.csv', usecols=[' id'])[' id'].values)
                        except: # if file doesn't exist
                            tweets_history = {}

                        # query_params['query'] = f'(from:{userid})'
                        output_tweets = main(search_url, query_params)
                        logging.info(f'Twitter URL: {search_url}')
                        logging.info(f'Params: {query_params}')

                        # If no data found:
                        if not output_tweets.get('data'):
                            save_csv = False
                            logging.warning(f'No data found for userId {userid}')
                            # continue
                            hast_next_token = False
                            break
                        next_token = {}
                        next_token['pagination_token'] = output_tweets['meta'].get('next_token')
                        if next_token['pagination_token']:
                            query_params['pagination_token'] = next_token['pagination_token']
                            hast_next_token = True
                        else:
                            hast_next_token = False

                        for tweet in output_tweets['data']:
                            
                            # Check if tweet id has already been collected
                            if tweet['id'] in tweets_history:
                                logging.warning(f'Reapeated tweet_id for {userid}: stopping historical data collection')
                                print(f'Reapeated tweet_id for {userid}: stopping historical data collection')
                                hast_next_token = False # break from while loop
                                break

                            # Get new columns
                            if not tweet.get('entities'):
                                entities_dict = parse_entities({}, {'hashtags': 'tag', 'urls': 'expanded_url'})
                            else:
                                entities_dict = parse_entities(tweet['entities'], {'hashtags': 'tag', 'urls': 'expanded_url'})

                            if not tweet.get('public_metrics'):
                                public_metrics_dict = expand_dict_object({}, 'public_metrics')
                            else:
                                public_metrics_dict = expand_dict_object(tweet['public_metrics'], 'public_metrics')

                            # Filter out unwanted columns
                            tweet = {key: tweet[key] for key in tweet.keys() if key in json_tweets_columns}

                            # Combine dicts
                            tweet = {**tweet, **entities_dict, **public_metrics_dict, **next_token, 'current_time': datetime.now()}
                            # tweet = sorted(tweet.items(), key=lambda kv: kv[1])
                            tweets_list_for_dataframe.append(tweet)

                    except Exception as err:
                        if err.args[0] == 429: # If ERROR = 429 (Too Many Requests, wait for retry) 
                            logging.info(f'Error 429. Sleep: {sleep}')
                            hast_next_token = False
                            time.sleep(sleep)
                            continue
                        else:
                            raise
                    break
                if save_csv:
                    tweets =  pd.DataFrame.from_records(tweets_list_for_dataframe, columns=tweet_columns)
                    tweets.to_csv(f'data/liked-tweets-{batch_of_users}.csv', mode='a', index=False, header=None)

/var/folders/sv/7czjs3z54yd54jz8j2ktmygc0000gn/T/ipykernel_6004/3436623643.py:114: DtypeWarning: Columns (6,7,8,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}/{batch_csv}")


1188283809826869248
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
914779140225675265
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0


KeyboardInterrupt: 

In [26]:
tweets_history = set(pd.read_csv('data/liked-tweets-0.csv', usecols=[' id'])[' id'].values)

In [28]:
1 if 1436866917508136960 in tweets_history else 0

1

In [48]:
tweets =  pd.DataFrame.from_records(tweets_list_for_dataframe)

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('data/data_stream/users-stream.csv')

In [11]:
df2 = df[['id', 'verified']].value_counts().reset_index()

In [15]:
df2['verified'].value_counts()

False    10860
True      3774
Name: verified, dtype: int64

In [16]:
3774/(10860+3774)

0.25789257892578926

In [50]:
tweets.to_csv(f'data/liked-tweets-{batch_of_users}-{date_str}-{time_str}.csv', index=False)